<a href="https://colab.research.google.com/github/franciscogarate/utils_ucm/blob/master/notebooks/Ejercicio_24_USP_B_Hipotesis_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/franciscogarate/utils_ucm

Cloning into 'utils_ucm'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 286 (delta 22), reused 4 (delta 2), pack-reused 239 (from 1)
Receiving objects: 100% (286/286), 7.56 MiB | 4.43 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [3]:
# Son datos ficticios de 1 millón de pólizas
df = pd.read_csv('./utils_ucm/data/siniestralidad_2024.csv')

print(df.head())
print(df.count())
print(df.Gto.sum())
print(df['Gto'].describe())

    Poliza    Gto
0    86921  12.79
1    99540  12.79
2   575008  12.79
3  1468724  66.42
4  1806378  44.28
Poliza    1000000
Gto       1000000
dtype: int64
75970777.12
count    1000000.000000
mean          75.970777
std          248.696378
min           12.790000
25%           15.740000
50%           29.520000
75%           59.530000
max        22807.450000
Name: Gto, dtype: float64


In [4]:
#Se establece una semilla para volver a replicar las mismas cifras si repito el proceso
np.random.seed(999)

n = df.Poliza.count()
b = 1000 #Con 1000 tarda 51 segundos

# Realizo un bootstrap con reemplazamiento (puedo seleccionar varias veces el mismo siniestro)
sampling = pd.DataFrame(data=np.zeros(b),columns=['z'])
for i in range(b):
	sample = df.sample(n, replace=True)
	summ = sample['Gto'].sum()
	sampling.loc[i, 'z'] = summ

sampling['ln'] = np.log(sampling['z'])

boot_mean = sampling['ln'].mean()
boot_std = sampling['ln'].std()
#print(sampling)
print('Media muestral: ', sampling['z'].mean())
# A utilizar para la demostracion de la hipotesis 2:
print('Var. muestral: ',  sampling['z'].var())

Media muestral:  75975309.09426
Var. muestral:  61034918717.58467


In [6]:
# Variables candidatas LN(mu,sigma)
print(boot_mean, boot_std)
mu=boot_mean
sigma=boot_std

18.145913682979504 0.0032519848074350184


In [7]:
lognormal = st.lognorm(s=sigma,scale=np.exp(mu))

# Kolmogorov-Smirnov test
ks_statistic, p_value = st.kstest(sampling['z'], lognormal.cdf)

print(f"Kolmogorov-Smirnov statistic: {ks_statistic}")
print(f"P-value: {p_value}")

alpha = 0.05  # Significance level

if p_value > alpha:
    print("The data follows a lognormal distribution (fail to reject H0)")
else:
    print("The data does not follow a lognormal distribution (reject H0)")

Kolmogorov-Smirnov statistic: 0.024716016701270382
P-value: 0.5657701734695861
The data follows a lognormal distribution (fail to reject H0)
